In [2]:
#Import Libraries

import torch
import tqdm as notebook_tqdm
import json
import csv
import gensim
from gensim.models import Word2Vec
import numpy as np

In [6]:
#Open Training Dataset
file =  open("msr_paraphrase_train.txt", "r", encoding='utf-8')
file_temp = file.readlines()
training_raw_list = []
for line in file_temp:
    training_raw_list.append(line.split('\t'))

In [9]:
#Training data initial processing
training_list1 = []
training_list2 = []
training_labels = []
training_labels_rev = []
for i in range(1, len(training_raw_list)):
    temp_label = training_raw_list[i][0]
    if int(temp_label) == 0:
        training_labels.append(0)
        training_labels_rev.append(1)
    elif int(temp_label) == 1:
        training_labels.append(1)
        training_labels_rev.append(0)
        
    temp_sent1 = training_raw_list[i][3]
    temp_sent2 = training_raw_list[i][4]
    temp_sent2 = temp_sent2.replace("\n", "")
    training_list1.append(temp_sent1)
    training_list2.append(temp_sent2)


In [10]:
print(len(training_list1))
print(len(training_list2))
print(len(training_labels))

4076
4076
4076


In [331]:
#Training data create 2 text files for CoreNLP processing
file1 = open("training_list1.txt", "w")
file2 = open("training_list2.txt", "w")
for element in training_list1:
    file1.write('%s\n' % element)
for element in training_list2:
    file2.write('%s\n' % element)
file1.close()
file2.close()

In [332]:
file1 = open("training_list1.txt", "r")
file2 = open("training_list2.txt", "r")
temp1 = file1.readlines()
temp2 = file2.readlines()
print(len(temp1))
print(len(temp2))
file1.close()
file2.close()

4076
4076


In [13]:
#Open Test Dataset
file =  open("msr_paraphrase_test.txt", "r", encoding='utf-8')
file_temp = file.readlines()
test_raw_list = []
for line in file_temp:
    test_raw_list.append(line.split('\t'))
file.close()

In [15]:
#Test data initial processing
test_list1 = []
test_list2 = [] 
test_labels = []
test_labels_rev = []
for i in range(1, len(test_raw_list)):
    temp_label = test_raw_list[i][0]
    if int(temp_label) == 0:
        test_labels.append(0)
        test_labels_rev.append(1)
    elif int(temp_label) == 1:
        test_labels.append(1)
        test_labels_rev.append(0)
        
    temp_sent1 = test_raw_list[i][3]
    temp_sent2 = test_raw_list[i][4]
    temp_sent2 = temp_sent2.replace("\n", "")
    test_list1.append(temp_sent1)
    test_list2.append(temp_sent2)


In [16]:
print(len(test_list1))
print(len(test_list2))
print(len(test_labels))

1725
1725
1725


In [17]:
#Test data create 2 text files for CoreNLP processing
file1 = open("test_list1.txt", "w")
file2 = open("test_list2.txt", "w")
for element in test_list1:
    file1.write('%s\n' % element)
for element in test_list2:
    file2.write('%s\n' % element)
file1.close()
file2.close()

In [18]:
#Process Training and Test text files with CoreNLP --> Output JSON files
#CoreNLP Command
#java -cp "*" -Xmx8g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators "tokenize,ssplit,pos,lemma,ner" -tokenize.whitespace=true -ssplit.eolonly -file training_list2.txt -outputFormat json

In [19]:
#Load Training JSONs Createed by CoreNLP
with open("training_list1.json", "r") as read_file1:
    training_json1 = json.load(read_file1)
with open("training_list2.json", "r") as read_file2:
    training_json2 = json.load(read_file2)
read_file1.close()
read_file2.close()

In [22]:
print(len(training_json1['sentences']))
print(len(training_json2['sentences']))
print(len(test_json1['sentences']))
print(len(test_json2['sentences']))

4076
4076
1725
1725


In [21]:
#Load Test JSONs Created by CoreNLP
with open("test_list1.json", "r") as read_file3:
    test_json1 = json.load(read_file3)
with open("test_list2.json", "r") as read_file4:
    test_json2 = json.load(read_file4)
read_file3.close()
read_file4.close()

In [23]:
core_words_train1 = []
core_words_train2 = []
max_sen_length_train = 0

core_words_test1 = []
core_words_test2 = []
max_sen_length_test = 0

In [24]:
#Create training sets core words lists based on CoreNLP parsing
for i in range(len(training_json1['sentences'])):
    temp_list = []
    for j in training_json1['sentences'][i]['tokens']:
        if 'NN' in j['pos'] or 'VB' in j['pos']:
            temp_list.append(j['originalText'].lower())
    core_words_train1.append(temp_list)
    if len(temp_list) > max_sen_length_train:
        max_sen_length_train = len(temp_list)
        
for i in range(len(training_json2['sentences'])):
    temp_list = []
    for j in training_json2['sentences'][i]['tokens']:
        if 'NN' in j['pos'] or 'VB' in j['pos']:
            temp_list.append(j['originalText'].lower())
    core_words_train2.append(temp_list)
    if len(temp_list) > max_sen_length_train:
        max_sen_length_train = len(temp_list)
        

In [25]:
print(len(core_words_train1))
print(len(core_words_train2))
print(max_sen_length_train)

4076
4076
22


In [26]:
#Create test sets core words lists based on CoreNLP parsing
for i in range(len(test_json1['sentences'])):
    temp_list = []
    for j in test_json1['sentences'][i]['tokens']:
        if 'NN' in j['pos'] or 'VB' in j['pos']:
            temp_list.append(j['originalText'].lower())
    core_words_test1.append(temp_list)
    if len(temp_list) > max_sen_length_test:
        max_sen_length_test = len(temp_list)
        
for i in range(len(test_json2['sentences'])):
    temp_list = []
    for j in test_json2['sentences'][i]['tokens']:
        if 'NN' in j['pos'] or 'VB' in j['pos']:
            temp_list.append(j['originalText'].lower())
    core_words_test2.append(temp_list)
    if len(temp_list) > max_sen_length_test:
        max_sen_length_test = len(temp_list)

In [27]:
print(len(core_words_test1))
print(len(core_words_test2))
print(max_sen_length_test)

1725
1725
22


In [28]:
#Combine train and test words to create training list for Word2Vec
word2vec_masterlist = []
for line in core_words_train1:
    word2vec_masterlist.append(line)
for line in core_words_train2:
    word2vec_masterlist.append(line)
for line in core_words_test1:
    word2vec_masterlist.append(line)
for line in core_words_test2:
    word2vec_masterlist.append(line)

In [30]:
len(word2vec_masterlist)

11602

In [31]:
#Train Word2Vec model
wv_model = gensim.models.Word2Vec(word2vec_masterlist, vector_size=50, workers=4, min_count=1)

In [33]:
train_tensors1 = []
train_tensors2 = []
test_tensors1 = []
test_tensors2 = []
zero_mask = torch.zeros(50)
zero_mask = torch.reshape(zero_mask, (-1,1))

In [34]:
#Create training sentence tensors
for sen in core_words_train1:
    temp_list = []
    for word in sen:
        w_vec = torch.tensor(wv_model.wv[word])
        w_vec = torch.reshape(w_vec, (-1,1))
        temp_list.append(w_vec)
    if len(temp_list) < max_sen_length_train:
        for i in range(len(temp_list), max_sen_length_train):
            temp_list.append(zero_mask)
    temp_stack = torch.stack(temp_list, dim = 0)
    temp_stack = torch.squeeze(temp_stack)
    temp_stack = torch.unsqueeze(temp_stack, 0)
    train_tensors1.append(temp_stack)
for sen in core_words_train2:
    temp_list = []
    for word in sen:
        w_vec = torch.tensor(wv_model.wv[word])
        w_vec = torch.reshape(w_vec, (-1,1))
        temp_list.append(w_vec)
    if len(temp_list) < max_sen_length_train:
        for i in range(len(temp_list), max_sen_length_train):
            temp_list.append(zero_mask)
    temp_stack = torch.stack(temp_list, dim = 0)
    temp_stack = torch.squeeze(temp_stack)
    temp_stack = torch.unsqueeze(temp_stack, 0)
    train_tensors2.append(temp_stack)

In [35]:
#Create test sentence tensors
for sen in core_words_test1:
    temp_list = []
    for word in sen:
        w_vec = torch.tensor(wv_model.wv[word])
        w_vec = torch.reshape(w_vec, (-1,1))
        temp_list.append(w_vec)
    if len(temp_list) < max_sen_length_test:
        for i in range(len(temp_list), max_sen_length_test):
            temp_list.append(zero_mask)
    temp_stack = torch.stack(temp_list, dim = 0)
    temp_stack = torch.squeeze(temp_stack)
    temp_stack = torch.unsqueeze(temp_stack, 0)
    test_tensors1.append(temp_stack)
for sen in core_words_test2:
    temp_list = []
    for word in sen:
        w_vec = torch.tensor(wv_model.wv[word])
        w_vec = torch.reshape(w_vec, (-1,1))
        temp_list.append(w_vec)
    if len(temp_list) < max_sen_length_test:
        for i in range(len(temp_list), max_sen_length_test):
            temp_list.append(zero_mask)
    temp_stack = torch.stack(temp_list, dim = 0)
    temp_stack = torch.squeeze(temp_stack)
    temp_stack = torch.unsqueeze(temp_stack, 0)
    test_tensors2.append(temp_stack)

In [36]:
#Create Training Master Sets
train_master_set1 = torch.stack(train_tensors1)
train_master_set2 = torch.stack(train_tensors2)

In [37]:
train_master_set1.shape

torch.Size([4076, 1, 22, 50])

In [38]:
#Create Training Master Labels
train_master_labels = torch.tensor(training_labels).float()
train_master_labels_rev = torch.tensor(training_labels_rev).float()

In [54]:
#Create Training Set Subsets - batch size = 64
train_set1 = []
train_set2 = []
split_train1 = np.array_split(train_tensors1, 64)
split_train2 = np.array_split(train_tensors2, 64)
for i in range(64):
    temp1 = torch.stack(list(split_train1[i]))
    train_set1.append(temp1)
    temp2 = torch.stack(list(split_train2[i]))
    train_set2.append(temp2)

In [57]:
#Create Train Label Sets
train_label_sets = []
split_train_labels = np.array_split(training_labels, 64)
for i in split_train_labels:
    train_label_sets.append(torch.tensor(i).float())


In [58]:
#Create Test Sets
test_set1 = []
test_set2 = []
split_test1 = np.array_split(test_tensors1, 27)
split_test2 = np.array_split(test_tensors2, 27)
for i in range(27):
    temp1 = torch.stack(list(split_test1[i]))
    test_set1.append(temp1)
    temp2 = torch.stack(list(split_test2[i]))
    test_set2.append(temp2)

In [59]:
#Create Test Master Sets
test_master_set1 = torch.stack(test_tensors1)
test_master_set2 = torch.stack(test_tensors2)

In [60]:
test_master_set1.shape

torch.Size([1725, 1, 22, 50])

In [61]:
#Create Test Master Labels
test_master_labels = torch.tensor(test_labels).float()

In [62]:
#Create Test Label Sets
test_label_sets = []
split_test_labels = np.array_split(test_labels, 27)
for i in split_test_labels:
    test_label_sets.append(torch.tensor(i).float())

In [63]:
#Import Pytorch Neural Network Packages

import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

In [64]:
from numpy import array  
from numpy.linalg import norm

In [65]:
#K-max pooling Code

def kmax_pooling(x, dim, k):
    index = torch.topk(x, k, dim = dim)[1].sort(dim = dim)[0]
    return x.gather(dim, index)

In [324]:
#Define Convolutional Neural Network

class CNN1(nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 5, 3, padding = 2)
        self.conv2 = nn.Conv2d(5, 25, 3, padding = 2)
        self.fc1 = nn.Linear(4050, 500)
        self.fc2 = nn.Linear(500, 25)
        self.pwd = nn.PairwiseDistance(p=1)

    def forward(self, x1, x2):
        
        x1 = F.relu(self.conv1(x1))
        x1 = kmax_pooling(x1, 2, 3)

        x1 = F.relu(self.conv2(x1))
        x1 = kmax_pooling(x1, 2, 3)
        
        x1 = torch.flatten(x1, 1)
        x1 = self.fc1(x1)
        x1 = self.fc2(x1)
        
        
        x2 = F.relu(self.conv1(x2))
        x2 = kmax_pooling(x2, 2, 3)

        x2 = F.relu(self.conv2(x2))
        x2 = kmax_pooling(x2, 2, 3)    
        
        x2 = torch.flatten(x2, 1)
        x2 = self.fc1(x2)
        x2 = self.fc2(x2)
        
        
        result1 = self.pwd(x1, x2)
        
        result2 = torch.exp(-result1)

        return result2

In [325]:
#Create CNN Model Instance

model = CNN1()

#Set evaluation criterion as Mean Square Error Loss

criterion = nn.MSELoss()

#Define optimizer as Stochastic Gradient Descent
optimizer = torch.optim.SGD(model.parameters(), lr = np.exp(-1))

In [326]:
n_epochs = 15
def train_model(model, train_data1, train_data2, training_labels, n_epoch=n_epochs, optimizer=optimizer, criterion=criterion):
    import torch.optim as optim

    model.train()
    
    for epoch in range(n_epoch):
        curr_epoch_loss = []
        for i in range(64):
            train_temp1 = train_data1[i]
            train_temp2 = train_data2[i]
            
            labels_temp = training_labels[i]
            
            y_hat = model(train_temp1, train_temp2)
            
            y_hat = torch.reshape(y_hat, (-1,1))

            labels_temp = torch.reshape(labels_temp, (-1,1))
        
            loss = criterion(y_hat, labels_temp)
            
            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()
        
            curr_epoch_loss.append(loss.cpu().data.numpy())
        
        print(f"Epoch {epoch}: curr_epoch_loss={np.mean(curr_epoch_loss)}")

    return model


In [327]:
#Train Model

seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

model = train_model(model, train_set1, train_set2, train_label_sets)

Epoch 0: curr_epoch_loss=0.22620275616645813
Epoch 1: curr_epoch_loss=0.21410632133483887
Epoch 2: curr_epoch_loss=0.20904117822647095
Epoch 3: curr_epoch_loss=0.20269352197647095
Epoch 4: curr_epoch_loss=0.2019137591123581
Epoch 5: curr_epoch_loss=0.19908171892166138
Epoch 6: curr_epoch_loss=0.19531476497650146
Epoch 7: curr_epoch_loss=0.19070246815681458
Epoch 8: curr_epoch_loss=0.1871507465839386
Epoch 9: curr_epoch_loss=0.1872679591178894
Epoch 10: curr_epoch_loss=0.18624259531497955
Epoch 11: curr_epoch_loss=0.18555477261543274
Epoch 12: curr_epoch_loss=0.18170391023159027
Epoch 13: curr_epoch_loss=0.1830725371837616
Epoch 14: curr_epoch_loss=0.17777256667613983


In [328]:
#Test Model Function
def eval_model(model, test_data1, test_data2, target_test):

    model.eval()
    Y_pred = []
    Y_test = []

        # your code here
        
    y_hat = model(test_data1, test_data2)
        
    y_pred = (y_hat > 0.5).type(torch.float)
    
    Y_pred = torch.reshape(y_pred, (-1,1))

    Y_test = torch.reshape(target_test, (-1,1))
        

    return Y_pred, Y_test

In [329]:
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [330]:
#Test Model, Calculate Accuracy and F1 Score

y_pred, y_true = eval_model(model, test_master_set1, test_master_set2, test_master_labels)
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print(("Validation Accuracy: " + str(acc)))
print(("Validation F1: " + str(f1)))

Validation Accuracy: 0.673623188405797
Validation F1: 0.7944505293902884
